In [7]:
import numpy as np
import os
from pynwb import NWBFile, NWBHDF5IO, image
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, ImagingPlane
from pynwb.file import Subject
from pynwb.device import Device
import math

import statistics
import pandas as pd
import h5py #Provides methods to supplement NWB
import datajoint as dj #
import csv 
from datetime import datetime
from dateutil import tz
from pynwb import NWBHDF5IO
from matplotlib import pylab as plt

def connect_to_dj():
    dj.config['database.host'] = 'spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com' 
    dj.config['database.user'] = 'MarikeReimer'
    dj.config['database.password'] = 'uqHKL3YLMCG0'
    dj.conn()

    #Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
    current_schema = dj.schema('MarikeReimer', locals())
    return current_schema


schema = connect_to_dj()
print(schema)
schema

#Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
schema = dj.schema('MarikeReimer', locals())
schema


#Define Mouse table
@schema
class Mouse(dj.Manual):
    definition = """
    subject_id: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    genotype: enum('B6', 'BalbC', 'Unknown', 'Thy1-YFP')
    sex: enum('M', 'F', 'Unknown')
    species: varchar(128)
    strain: varchar(128)
    """

@schema
class Session(dj.Manual):
    definition = """
    ->Mouse
    identifier: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    surgery: varchar(128)
    pharmacology: varchar(128)
    """


@schema
class Dendrite(dj.Manual):
    definition = """
    ->Session
    dendrite_id: int                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    soma_center_point: longblob
    proximal_dendrite_length: float
    medial_dendrite_length: float
    distal_dendrite_length: float
    """

@schema
class Image_segmentation(dj.Imported):
    definition = """
    ->Dendrite
    segmentation_name: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    length: float
    volume: float
    surface_area:float
    spine_type: enum('mushroom', 'thin', 'disconnected', 'stubby', 'strict_thin', 'U')
    center_of_mass: longblob
    """
    def make(self, key):
        subject_id = key['subject_id']
        identifier = key['identifier']

        nwbfile_to_read = 'C:/Users/meowm/OneDrive/TanLab/DataJointTesting/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb' #TODO: Remove hard coding
        #nwbfile_to_read = 'C:/Users/meowm/Downloads/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb'

        print(nwbfile_to_read)
        with NWBHDF5IO(nwbfile_to_read, 'r') as io:
            nwbfile = io.read()     
            for group in nwbfile.processing["SpineData"]["ImageSegmentation"].children[:]:
                #print(group.name)
                if group.name.startswith("Mushroom"):
                    spine_type = 'mushroom'
                elif group.name.startswith("Thin"):
                    spine_type = 'thin'
                elif group.name.startswith("Disconnected"):
                    spine_type = 'disconnected'
                elif group.name.startswith("Stubby"):
                    spine_type = 'stubby'
                elif group.name.startswith("Strict"):
                    spine_type = 'strict_thin'
                else:
                    spine_type = 'U'

                length = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].length.data[:]
                length = length[0]
                volume = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].volume.data[:]
                volume = volume[0]
                surface_area = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].surface_area.data[:]
                surface_area = surface_area[0]
                center_of_mass = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].center_of_mass.data[:]
                center_of_mass = center_of_mass[0]
                
                key['segmentation_name'] = group.name 
                key['length'] = length
                key['volume'] = volume
                key['surface_area'] = surface_area
                key['spine_type'] = spine_type
                key['center_of_mass'] = center_of_mass
                self.insert1(key)

@schema
class Distance_to_soma(dj.Computed):
    definition = """
    ->Image_segmentation
    ---
    distance_to_soma: float"""
    def make(self, key):
        center_of_mass = (Image_segmentation() & key).fetch1('center_of_mass')
        soma_center_point = (Dendrite() & key).fetch1('soma_center_point')
        distance_to_soma = math.dist(center_of_mass,soma_center_point)

        key['distance_to_soma'] = distance_to_soma
        self.insert1(key)

#Instantiate tables
mouse = Mouse()
session = Session()
dendrite = Dendrite()
image_segmentation = Image_segmentation()
distance_to_soma = Distance_to_soma()

print(distance_to_soma)

Schema `MarikeReimer`

*subject_id    *identifier    *dendrite_id   *segmentation_ distance_to_so
+------------+ +------------+ +------------+ +------------+ +------------+

 (Total: 0)



In [6]:
#drop all the tables
mouse.drop()

[2024-01-01 10:24:59,019][INFO]: `MarikeReimer`.`mouse` (1 tuples)
[2024-01-01 10:24:59,158][INFO]: `MarikeReimer`.`session` (1 tuples)
[2024-01-01 10:24:59,293][INFO]: `MarikeReimer`.`dendrite` (1 tuples)
[2024-01-01 10:24:59,429][INFO]: `MarikeReimer`.`_image_segmentation` (3 tuples)
[2024-01-01 10:24:59,562][INFO]: `MarikeReimer`.`__distance_to_soma` (3 tuples)
[2024-01-01 10:25:02,411][INFO]: Dropped table `MarikeReimer`.`__distance_to_soma`
[2024-01-01 10:25:02,738][INFO]: Dropped table `MarikeReimer`.`_image_segmentation`
[2024-01-01 10:25:03,050][INFO]: Dropped table `MarikeReimer`.`dendrite`
[2024-01-01 10:25:03,395][INFO]: Dropped table `MarikeReimer`.`session`
[2024-01-01 10:25:03,725][INFO]: Dropped table `MarikeReimer`.`mouse`
[2024-01-01 10:25:03,943][INFO]: Tables dropped. Restart kernel.


In [5]:
#For general load checking
mouse*session

subject_id Primary keys above the '---',identifier Primary keys above the '---',genotype,sex,species,strain,surgery,pharmacology
TestMouse,Dendrite1,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI


In [ ]:
#Check if a specific subject loaded
session & 'subject_id = "L882"'

In [ ]:
#Delete a Subject with an identifier - use after failing QA on a Blender file
(session & 'subject_id = "L882"' & 'identifier = "Dendrite4c"').delete()

In [ ]:
#Display info for a Subject with an identifier 

(session & image_segmentation & distance_to_soma & 'subject_id = "L831"' & 'identifier = "Dendrite1"')

In [ ]:
#Display info for a Subject with an identifier 

frame = pd.DataFrame((mouse&'subject_id = "L831"')*(session & 'identifier = "Dendrite3"')*image_segmentation)
#(session & image_segmentation & 'subject_id = "L736"' & 'identifier = "Dendrite1"'& distance_to_soma )
frame

In [ ]:
#Pivot tables in Excel exclude blank values, so I download this separately to make sure spineless dendrites don't get missed
frame = pd.DataFrame(mouse*session*dendrite)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/SubjList.csv")

In [ ]:
#Export pipeline to csv
frame = pd.DataFrame(mouse*session*dendrite*image_segmentation*distance_to_soma)
#pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/UnfilteredDiscData_Vsad_dendritestart.csv")
frame

In [ ]:
#View pipeline
#mouse*(session& 'surgery = "Contusion"')*dendrite*image_segmentation*distance_to_soma
mouse*(session)*dendrite*image_segmentation*distance_to_soma

In [ ]:
#Query for dataset 1.3+

frame = pd.DataFrame(mouse*session*dendrite*(image_segmentation & 'volume >= "0.01"' & 'volume <= "0.8"' &'length >= "0.2"' & 'length <= "3"')*distance_to_soma)
print(frame)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/DiscSpinedata_V1.5.csv")

In [ ]:
#Didn't meet criteria because too small

frame = pd.DataFrame(mouse*(session & 'pharmacology = "Vehicle 1% DMSO inject IP 4 weeks after SCI"')*dendrite*(image_segmentation & 'volume <= "0.01"' or 'length < "0.2" ' )*distance_to_soma)
#pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/toosmall.csv")
print(len(frame))

In [ ]:
#Didn't meet criteria because too big
frame = pd.DataFrame(mouse*(session & 'pharmacology = "Vehicle 1% DMSO inject IP 4 weeks after SCI"')*dendrite*(image_segmentation & 'volume > "0.8"' or 'length > "3"')*distance_to_soma)
#pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/toobig.csv")
print(len(frame))

In [ ]:
mouse